# Testing and Benchmarking

We load the best weights we've found, and load them in a model.

In [1]:
#Load best model weights
from keras.models import load_model
model = load_model('tmp/weights-32b-106.hdf5')

C:\Users\TWright\Anaconda3\envs\tensorflow-gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Unlike training we resize the images online here, it takes about 10 minutes.

In [2]:
import glob
from keras.preprocessing.image import img_to_array
from skimage.transform import resize
from skimage.io import imread

def preprocess_image(fname, crop=212, downsample=106):
    sample_img = imread(fname)
    crop_from = int(crop / 2)
    crop_to   = int(crop * 1.5)
    sample_img = sample_img.T[:, crop_from:crop_to, crop_from:crop_to]
    sample_img = resize(sample_img, (downsample, downsample, 3), mode='reflect').T
    return sample_img

def fetch_test_images():
    while 1:
        for fname in test_files:
            image = preprocess_image(fname)
            image = img_to_array(image)
            test = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
            yield (test)

In [3]:
test_files = glob.glob('../data/images_test_rev1/*.jpg')

Generate predictions.

In [4]:
predictions = model.predict_generator(fetch_test_images(),
                        steps=len(test_files),
                        max_queue_size=32,)
predictions.shape # assert (79975, 37)

(6, 37)

You can visually verify predictions, every three numbers should add up to 1.

In [6]:
predictions[0]

array([0.34669548, 0.64004695, 0.02467277, 0.15057456, 0.49103555,
       0.12518845, 0.3738037 , 0.2108857 , 0.2839931 , 0.04479966,
       0.20094246, 0.2365846 , 0.0420614 , 0.20211063, 0.7962315 ,
       0.07085016, 0.21115689, 0.07749238, 0.04369585, 0.01599406,
       0.03778622, 0.04182997, 0.06239795, 0.05049454, 0.00701428,
       0.09954717, 0.01944779, 0.04457118, 0.08770391, 0.08399535,
       0.0535618 , 0.02134417, 0.11054694, 0.01623139, 0.00815554,
       0.00795962, 0.08655441], dtype=float32)

Record all results into a csv file with the id and the predictions. The CSV file will be about 40MB big. We only load the all_zeros.csv file for its header.

In [25]:
from os import path

header = open('../data/all_zeros_benchmark.csv','r').readlines()[0]

with open('submission_1.csv','w') as outfile:
    outfile.write(header)
    for i, fname in enumerate(test_files):
        fname = test_files[i]
        file_id = path.split(fname)[-1] \
                      .split('.')[0]
        pred = predictions[i]
        outline = file_id + "," + ",".join([str(x) for x in pred])
        outfile.write(outline + "\n")